# Compare runs

In [26]:
import pdal 
import numpy as np
import matplotlib.pyplot as plt

from osgeo import ogr
import pyvista as pv
import os
import sys
from scipy.spatial import KDTree

from interessant import * # Bei Änderungen Kernel neu starten
sys.path.append("/home/riannek/code/gleis/gleisachse")
from algs.centralpoints import *

ogr.UseExceptions()

In [27]:
run24 = "/media/riannek/minimax/gleis/run24-2024-08-13"
run14 = "/media/riannek/minimax/gleis/run14-2024-08-14"

run24_tmp = "/media/riannek/minimax/gleis/temp_run24"
run14_tmp = "/media/riannek/minimax/gleis/temp_run14"

# Bahnsteig: 29; Gleis hohe Intensität: 11; Weiche B: 16; Unterirdischer Bhf: 20; Gleis weit abseits: 23; Betondeckel: 28; Zug run 14 A: 6; 
# Viele Gleise: 33; Anfang Weiche: 34; OLA gleiche H: 35; Y: 37; Auch viele Gleise: 43; Kreuzung: 44, 45; 47 Drei

key = list(interessant.keys())[34] 

filename = interessant[key]

print(key, filename)
plyfilename = filename.split(".")[0] + ".ply"

file_run24 = os.path.join(run24, filename)
file_run14 = os.path.join(run14, filename)
ply_run24 = os.path.join(run24_tmp, "candidates", plyfilename)
ply_run14 = os.path.join(run14_tmp, "candidates", plyfilename)

if not os.path.exists(file_run14):
    raise FileNotFoundError(filename)

Anfang Weiche 4481275_5357000.copc.laz


In [28]:
gauge = 1.435
railhead_width = 0.067

In [29]:
def get_seedpoints(points):
    low_intensity = points[points["Intensity"] < 14500]
    downsampling_pipeline = pdal.Filter("filters.sample", radius=s.seedpoint_distance).pipeline(low_intensity)
    downsampling_pipeline.execute()
    seed_points = downsampling_pipeline.arrays[0]
    return np.vstack((seed_points["X"], seed_points["Y"], seed_points["Z"])).T

In [30]:
def get_points(xyz, xyz_seed): 
    points = []
    tree = KDTree(xyz)
    # indices: ndarray (dtype object) with a list of indices for each seed point
    indices = tree.query_ball_point(xyz_seed, r=s.neighborhood_radius, workers=-1)

    seed_point_count = len(xyz_seed)

    for cluster, clustercenter, cluster_other, center_other, eigenvects in pair_generator(xyz, tree, indices, seed_point_count, gauge):
        transformmatrix = get_transformmatrix(cluster, cluster_other, clustercenter, center_other, eigenvects)
        representative_a, side_a  = representative_point(cluster, transformmatrix, railhead_width) 
        representative_b, side_b = representative_point(cluster_other, transformmatrix, railhead_width)

        if representative_a is None or representative_b is None:
            # Happens in rare cases at switches
            continue

        # Check if both are classified as the same side
        # but a zero and non zero value is not a problem
        if side_a * side_b < 0:
            # Update the cluster with lower confidence of the side estimation
            if np.abs(side_a) < np.abs(side_b):
                representative_a, side_a  = representative_point(cluster, transformmatrix, railhead_width, side=side_b)
            else:
                representative_b, side_b = representative_point(cluster_other, transformmatrix,  railhead_width, side=side_a)

        if representative_a is None or representative_b is None:
            continue

        central = get_central_point(representative_a, representative_b)

        points.append(central)
        points.append(representative_a)
        points.append(representative_b)

    return np.array(points)

In [31]:
def read_pointcloud(filename):
    pipeline = pdal.Pipeline([pdal.Reader(filename)])
    pipeline.execute()
    points = pipeline.arrays[0]
    xyz = np.vstack((points["X"], points["Y"], points["Z"])).T
    return xyz, points

In [32]:
# Read full point clouds
xyz_run24, _ =read_pointcloud(file_run24)
xyz_run14, _ =read_pointcloud(file_run14)

# Read the ply files
xyz_ply_run24, points_ply_run24 = read_pointcloud(ply_run24)
xyz_ply_run14, points_ply_run14 = read_pointcloud(ply_run14)

seed24 = get_seedpoints(points_ply_run24)
seed14 = get_seedpoints(points_ply_run14)

offset = xyz_run24.mean(axis=0)
xyz_run24 -= offset
xyz_run14 -= offset
xyz_ply_run24 -= offset
xyz_ply_run14 -= offset
seed24 -= offset
seed14 -= offset


cpoints24 = get_points(xyz_ply_run24, seed24)
cpoints14 = get_points(xyz_ply_run14, seed14)


In [33]:
pv24 = pv.PolyData(xyz_run24)
pv14 = pv.PolyData(xyz_run14)


pc24 = pv.PolyData(cpoints24)
pc14 = pv.PolyData(cpoints14)


p = pv.Plotter()
p.add_mesh(pv24, color="salmon",name="Run 24", opacity=0.5, point_size=1)
p.add_mesh(pv14, color="skyblue", name="Run 14", opacity=0.5, point_size=1)

p.add_mesh(pc24, color="red", name="Central points 24", point_size=5)
p.add_mesh(pc14, color="navy", name="Central points 14", point_size=5)

p.show()

Widget(value='<iframe src="http://localhost:36535/index.html?ui=P_0x7fbf03f8a4f0_4&reconnect=auto" class="pyvi…